In [63]:
from sklearn import preprocessing, compose, pipeline

In [8]:
import pandas as pd
import ugvc.filtering.variant_filtering_utils as vfu

In [10]:
from ugvc.filtering.variant_filtering_utils import *

In [106]:
df = pd.read_hdf("test/resources/system/test_train_models_pipeline/180801-UGAv3-26_chr1_1_5000000.comp.h5", key="chr1")

In [145]:
default_filler = impute.SimpleImputer(strategy="constant", fill_value=0)
tuple_filter = preprocessing.FunctionTransformer(tuple_break)
tuple_filter_second = preprocessing.FunctionTransformer(tuple_break_second)
left_motif_filter = preprocessing.FunctionTransformer(lambda x: pd.DataFrame(np.array([motif_encode_left(y) for y in x ]).reshape((-1,1)), index=x.index))

right_motif_filter = preprocessing.FunctionTransformer(lambda x: pd.DataFrame(np.array([motif_encode_right(y) for y in x ]).reshape((-1,1)), index=x.index))

def allele_encode_df(s):
    return pd.DataFrame(np.array([x[:2] for x in s]), index=s.index).applymap(allele_encode)
allele_filter = preprocessing.FunctionTransformer(allele_encode_df)

gt_filter = preprocessing.FunctionTransformer(gt_encode)
tuple_filter = preprocessing.FunctionTransformer( lambda x: 
                                                     pd.DataFrame(np.array( [tuple_break(y) for y in x ]).reshape((-1,1)), index=x.index))



In [146]:

    transform_list = [
        ("sor", default_filler,["sor"]),
        ("dp", default_filler, ["dp"]),
        ("inside_hmer_run", 'passthrough', ['inside_hmer_run']),
        ("close_to_hmer_run", 'passthrough', ['close_to_hmer_run']),
        ("hmer_indel_length", default_filler, ['hmer_indel_length']),
        ("indel_length", default_filler,['indel_length']),
        ("fs", default_filler,['fs']),
        ("qd", default_filler,['qd']),
        ("mq", default_filler,['mq']),
        ("an", default_filler,['an']),
        ("baseqranksum", default_filler,['baseqranksum']),
        ("excesshet", default_filler,['excesshet']),
        ("mqranksum", default_filler,['mqranksum']),
        ("readposranksum", default_filler,['readposranksum']),
        ("indel", 'passthrough',["indel"]),
        ("hmer_indel_nuc", preprocessing.OrdinalEncoder(), ["hmer_indel_nuc"]),
        ("left_motif", left_motif_filter, "left_motif"),
        ("right_motif", right_motif_filter, "right_motif"),
        ("cycleskip_status", preprocessing.OrdinalEncoder(), ["cycleskip_status"]),
        ("alleles", allele_filter, "alleles"),
        ("gc_content", default_filler, ['gc_content']),
        ("ac", tuple_filter, 'ac')
    ]



In [147]:
%pdb on
ct = compose.ColumnTransformer(transform_list).set_output(transform="pandas")
ct.fit(df)

Automatic pdb calling has been turned ON


/home/ilyas/anaconda3/envs/genomics.py3/lib/python3.8/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


ColumnTransformer(transformers=[('sor',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['sor']),
                                ('dp',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['dp']),
                                ('inside_hmer_run', 'passthrough',
                                 ['inside_hmer_run']),
                                ('close_to_hmer_run', 'passthrough',
                                 ['close_to_hmer_run']),
                                ('hmer_indel_length',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['hmer_inde...
                                 FunctionTransformer(func=<function <lambda> at 0x7fd1ceed28b0>),
                                 'right_motif'),
                                ('cycleskip_status', OrdinalEncoder(),
                                 ['cycleskip_status']),
                                ('alleles',
                                 FunctionTransformer(func=<function allele_encode_df at 0x7fd1ceed2af0>),
                                 'alleles'),
                                ('gc_content',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 ['gc_content']),
                                ('ac',
                                 FunctionTransformer(func=<function <lambda> at 0x7fd1ceed2700>),
                                 'ac')])

In [128]:
dd = ct.transform(df)

In [129]:
dd.columns

Index(['sor__sor', 'dp__dp', 'inside_hmer_run__inside_hmer_run',
       'close_to_hmer_run__close_to_hmer_run',
       'hmer_indel_length__hmer_indel_length', 'indel_length__indel_length',
       'fs__fs', 'qd__qd', 'mq__mq', 'an__an', 'baseqranksum__baseqranksum',
       'excesshet__excesshet', 'mqranksum__mqranksum',
       'readposranksum__readposranksum', 'indel__indel',
       'hmer_indel_nuc__hmer_indel_nuc', 'left_motif__0', 'right_motif__0',
       'cycleskip_status__cycleskip_status', 'alleles__0', 'alleles__1',
       'gc_content__gc_content'],
      dtype='object')

In [130]:
%pdb off

Automatic pdb calling has been turned OFF
